
### Universidade Federal de Pernambuco - IF998 (Solução em Mineração de Dados)

### Professor: Leandro Almeida - lma3@cin.ufpe.br

### Alunos: 
- Daniel Moraes Costa Andrade - dmca@cin.ufpe.br
- Gustavo Prazeres Paz do Nascimento - gppn@cin.ufpe.br
- Vinicius Luiz da Silva Franca - vlsf2@cin.ufpe.br

## 08 - Ensemble Learning- Realizar a implementação do bagging, boosting e stacking

### -------------------------------------------------------

Realizar experimento com a implementação de comitês bagging, boosting e stacking para base de dados do Credit Approval Data Set. Os modelos base para formação dos comitês poderão ser KNN, SVM, Árvore de Decisão e/ou Redes Neurais Artificiais. Realizar uma busca de tal modo que os comitês apresentem desempenho superior ao desempenho solo dos classificadores. Comparar o desempenho com os modelos de SVM desenvolvidos na última missão. Apresentar análise, os principais desafios encontrados para a construção dos comitês e conclusões. Enviar o relatório e scripts.

# Importando bibliotecas

In [ ]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importando os dados

In [ ]:

dt_parameters = ['Male','Age','Debt','Married','BankCustomer','EducationLevel','Ethnicity','YearsEmployed','PriorDefault','Employed','CreditScore','DriversLicense','Citizen','ZipCode','Income','Approved']

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"

dataset = pd.read_csv(url, delimiter=",", names=dt_parameters)

no_registros = dataset.shape[0]
no_atributos = dataset.shape[1]

print(f'Carregado com sucesso! \n\nO Dataset possui {no_registros} registros com {no_atributos} atributos cada\n')

print('Exibindo os 10 primeiros registros:')
dataset.head(10)

Carregado com sucesso! 

O Dataset possui 690 registros com 16 atributos cada

Exibindo os 10 primeiros registros:


,Male,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,b,30.83,0.000,u,g,w,v,1.250,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.040,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.500,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.750,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.710,t,f,0,f,s,00120,0,+
5,b,32.08,4.000,u,g,m,v,2.500,t,f,0,t,g,00360,0,+
6,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,00164,31285,+
7,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,00080,1349,+
8,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,00180,314,+
9,b,42.50,4.915,y,p,w,v,3.165,t,f,0,t,g,00052,1442,+


In [ ]:
dataset.shape

(690, 16)

# Descrevendo o Dataset

Pandas describe() é usado para visualizar alguns detalhes estatísticos básicos como percentil, média, std etc. de um quadro de dados ou uma série de `valores numéricos`

In [ ]:
dataset.describe([.20, .40, .60, .80])

,Debt,YearsEmployed,CreditScore,Income
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
20%,0.750000,0.125000,0.00000,0.000000
40%,1.934000,0.500000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
60%,4.016000,1.500000,1.00000,50.000000
80%,9.432000,3.500000,4.00000,600.000000
max,28.000000,28.500000,67.00000,100000.000000


# Pré-Processamento 

**1. Limpeza de dados**


In [ ]:
dataset[dataset.eq("?").any(1)] # Verificando todas as linhas que possuem os valores nulos (interrogações)

,Male,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
71,b,34.83,4.000,u,g,d,bb,12.500,t,f,0,t,g,?,0,-
83,a,?,3.500,u,g,d,v,3.000,t,f,0,t,g,00300,0,-
86,b,?,0.375,u,g,d,v,0.875,t,f,0,t,s,00928,0,-
92,b,?,5.000,y,p,aa,v,8.500,t,f,0,f,g,00000,0,-
97,b,?,0.500,u,g,c,bb,0.835,t,f,0,t,s,00320,0,-
202,b,24.83,2.750,u,g,c,v,2.250,t,t,6,f,g,?,600,+
206,a,71.58,0.000,?,?,?,?,0.000,f,f,0,f,p,?,0,+
243,a,18.75,7.500,u,g,q,v,2.710,t,t,5,f,g,?,26726,+
248,?,24.50,12.750,u,g,c,bb,4.750,t,t,2,f,g,00073,444,+
254,b,?,0.625,u,g,k,v,0.250,f,f,0,f,g,00380,2010,-


## Substituição de variáveis categóricas

Ao mostrar as informações do dataset, vemos que a coluna ZipCode possui valores numéricos está com a tipagem errada, vamos alterar pois este atributo apresenta dados bem distribuídos que são interessantes para exploração

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Male            690 non-null    object 
 1   Age             690 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         690 non-null    object 
 4   BankCustomer    690 non-null    object 
 5   EducationLevel  690 non-null    object 
 6   Ethnicity       690 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  ZipCode         690 non-null    object 
 14  Income          690 non-null    int64  
 15  Approved        690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


Vamos inserir o valor da média do atributo Age nos valores inconsistentes. Essa é uma forma de tratar dados inconsistentes sem que interfira negativamente nas análises posteriores.

In [ ]:
# CONVERTENDO COLUNA Age EM NUMÉRICO

dataset_Age = dataset[dataset['Age'] == '?'] #Pegar valores inconsistentes em Age
dataset    =  dataset[dataset['Age'] != '?'] #Pegar valores consistentes em Age

dataset.Age = dataset.Age.astype(np.float32) #Convertendo atributo Age para numérico
dataset_Age.loc[(dataset_Age['Age'] == '?'), 'Age'] = dataset['Age'].mean() #Inserindo a média de Age nas instâncias inconsistentes
dataset_Age.Age = dataset_Age.Age.astype(np.float32)
dataset = pd.concat([dataset, dataset_Age], axis=0) #Concatenando e finalizando ajuste

dataset.describe()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Age,Debt,YearsEmployed,CreditScore,Income
count,690.000000,690.000000,690.000000,690.00000,690.000000
mean,31.568172,4.758725,2.223406,2.40000,1017.385507
std,11.853271,4.978163,3.346513,4.86294,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000
25%,22.670000,1.000000,0.165000,0.00000,0.000000
50%,28.625000,2.750000,1.000000,0.00000,5.000000
75%,37.707500,7.207500,2.625000,3.00000,395.500000
max,80.250000,28.000000,28.500000,67.00000,100000.000000


In [ ]:
dataset.sort_index(ascending=True, inplace=True)
dataset

,Male,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,b,30.830000,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.669998,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.500000,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.830000,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.170000,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.080000,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.670000,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.250000,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.920000,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


In [ ]:
dataset.replace('?', np.nan, inplace = True) # Substituindo as interrogações por um valor numérico nulo
dataset.isnull().sum() # Validando que as interrogações foram de fato substituídas por valores nulos

Male              12
Age                0
Debt               0
Married            6
BankCustomer       6
EducationLevel     9
Ethnicity          9
YearsEmployed      0
PriorDefault       0
Employed           0
CreditScore        0
DriversLicense     0
Citizen            0
ZipCode           13
Income             0
Approved           0
dtype: int64

In [ ]:
dataset.shape

(690, 16)

In [ ]:
dataset = dataset.dropna()
dataset.isnull().sum()

Male              0
Age               0
Debt              0
Married           0
BankCustomer      0
EducationLevel    0
Ethnicity         0
YearsEmployed     0
PriorDefault      0
Employed          0
CreditScore       0
DriversLicense    0
Citizen           0
ZipCode           0
Income            0
Approved          0
dtype: int64

In [ ]:
dataset.reset_index(drop=True, inplace=True)
dataset

,Male,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,b,30.830000,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.669998,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.500000,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.830000,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.170000,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,b,21.080000,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
660,a,22.670000,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
661,a,25.250000,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
662,b,17.920000,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


In [ ]:
print(f' Nosso dataset agora possui {dataset.shape[0]} linhas.')

 Nosso dataset agora possui 664 linhas.


Substituindo nossa classe Approved por booleano

In [ ]:
dataset['Approved'].replace(['+', '-'],
                        [1, 0], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Já nesse outro Boxplot agrupado pela coluna [Approved], ao analisar o 3º quartil e o limite superior da classe [+], podemos notar que o grupo da classe [+] possuem valores consideravelmente maiores em comparação com o grupo [-].

# Transformação de Dados

## Normalização dos dados


In [ ]:
dataset.sort_index(ascending=True,inplace=True)
dataset

,Male,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,b,30.830000,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,1
1,a,58.669998,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,1
2,a,24.500000,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,1
3,b,27.830000,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,1
4,b,20.170000,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,b,21.080000,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,0
660,a,22.670000,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,0
661,a,25.250000,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,0
662,b,17.920000,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,0


In [ ]:
# DADOS NUMÉRICOS

def mean_norm(dataset):
    return dataset.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

numerics_dataset = dataset[['Age', 'Debt', 'YearsEmployed', 'CreditScore', 'Income']]
numerics_dataset = mean_norm(numerics_dataset)
numerics_dataset.head()


,Age,Debt,YearsEmployed,CreditScore,Income
0,-0.057487,-0.958978,-0.300443,-0.297501,-0.192583
1,2.313963,-0.067795,0.231090,0.715529,-0.085122
2,-0.596685,-0.859070,-0.226206,-0.500108,-0.034462
3,-0.313031,-0.651260,0.441921,0.512923,-0.192007
4,-0.965521,0.164992,-0.163848,-0.500108,-0.192583


In [ ]:
numerics_dataset.shape

(664, 5)

In [ ]:
## DADOS NUMÉRICOS NORMALIZADOS
dataset[['Age', 'Debt', 'YearsEmployed', 'CreditScore', 'Income']] = numerics_dataset[['Age', 'Debt', 'YearsEmployed', 'CreditScore', 'Income']]
dataset

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Male,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved
0,b,-0.057487,-0.958978,u,g,w,v,-0.300443,t,t,-0.297501,f,g,00202,-0.192583,1
1,a,2.313963,-0.067795,u,g,q,h,0.231090,t,t,0.715529,f,g,00043,-0.085122,1
2,a,-0.596685,-0.859070,u,g,q,h,-0.226206,t,f,-0.500108,f,g,00280,-0.034462,1
3,b,-0.313031,-0.651260,u,g,w,v,0.441921,t,t,0.512923,t,g,00100,-0.192007,1
4,b,-0.965521,0.164992,u,g,w,v,-0.163848,t,f,-0.500108,f,s,00120,-0.192583,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,b,-0.888006,1.056176,y,p,e,h,-0.300443,f,f,-0.500108,f,g,00260,-0.192583,0
660,a,-0.752567,-0.809116,u,g,c,v,-0.077733,f,t,-0.094895,t,g,00200,-0.116976,0
661,a,-0.532799,1.738550,y,p,ff,ff,-0.077733,f,t,-0.297501,t,g,00200,-0.192391,0
662,b,-1.157179,-0.918016,u,g,aa,v,-0.659746,f,f,-0.500108,f,g,00280,-0.048662,0


## Redução de dados e Métodos para seleção de atributos

In [ ]:
dataset.nunique()

Male                2
Age               341
Debt              213
Married             3
BankCustomer        3
EducationLevel     14
Ethnicity           9
YearsEmployed     131
PriorDefault        2
Employed            2
CreditScore        23
DriversLicense      2
Citizen             3
ZipCode           166
Income            232
Approved            2
dtype: int64

In [ ]:
# CONVERTENDO VALORES CATEGÓRICOS EM NUMÉRICOS (BINÁRIOS)
from sklearn.preprocessing import OneHotEncoder

# SELEÇÃO DE DADOS CATEGÓRICOS
attr_drop=  ['EducationLevel', 'Ethnicity', 'Citizen', 'ZipCode']
for attr in attr_drop:
  dataset.drop(attr, axis=1, inplace=True)
dataset.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Male,Age,Debt,Married,BankCustomer,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Income,Approved
0,b,-0.057487,-0.958978,u,g,-0.300443,t,t,-0.297501,f,-0.192583,1
1,a,2.313963,-0.067795,u,g,0.231090,t,t,0.715529,f,-0.085122,1
2,a,-0.596685,-0.859070,u,g,-0.226206,t,f,-0.500108,f,-0.034462,1
3,b,-0.313031,-0.651260,u,g,0.441921,t,t,0.512923,t,-0.192007,1
4,b,-0.965521,0.164992,u,g,-0.163848,t,f,-0.500108,f,-0.192583,1


In [ ]:
# Armazenando atributo de classe para futura análise do PCA
class_dataset = dataset[['Approved']]

É possível verificar que os dados estão normalizados

In [ ]:
class_dataset.value_counts()

Approved
0           366
1           298
dtype: int64

In [ ]:
print("selecionar apenas as colunas que sao do tipo objeto/categorigos\n")
categorical_dataset = dataset.select_dtypes(include=['object']).copy()

onehotenconder = OneHotEncoder() 
categorical_dataset_ohe = onehotenconder.fit_transform(categorical_dataset).toarray()
print(f'Os {dataset.shape[1]} atributos categóricos foram transformados em {categorical_dataset.shape[1]} Colunas')

selecionar apenas as colunas que sao do tipo objeto/categorigos

Os 12 atributos categóricos foram transformados em 6 Colunas


In [ ]:
feature_names_ohe = onehotenconder.get_feature_names(categorical_dataset.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
categorical_dataset_ohe = pd.DataFrame(categorical_dataset_ohe, columns=feature_names_ohe)

categorical_dataset_ohe

,Male_a,Male_b,Married_l,Married_u,Married_y,BankCustomer_g,BankCustomer_gg,BankCustomer_p,PriorDefault_f,PriorDefault_t,Employed_f,Employed_t,DriversLicense_f,DriversLicense_t
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
660,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
661,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
662,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


Como os algoritmos não realizam predições com atributos categóricos, o estado final do Dataset deverá ser este, com todos os atributos numéricos para que os algoritmos possam realizar predições eficientes. Com isso, este dataset se torna inelegível para análise humana, fazendo com que os dados não tenham um sentido aparente.

In [ ]:
# ESTADO FINAL DO DATASET

numerics_dataset = dataset[['Age', 'Debt', 'YearsEmployed', 'CreditScore', 'Income']]
dataset = pd.concat([categorical_dataset_ohe, numerics_dataset], axis=1) #Uma coluna ao lado da outra
dataset

,Male_a,Male_b,Married_l,Married_u,Married_y,BankCustomer_g,BankCustomer_gg,BankCustomer_p,PriorDefault_f,PriorDefault_t,Employed_f,Employed_t,DriversLicense_f,DriversLicense_t,Age,Debt,YearsEmployed,CreditScore,Income
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-0.057487,-0.958978,-0.300443,-0.297501,-0.192583
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.313963,-0.067795,0.231090,0.715529,-0.085122
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,-0.596685,-0.859070,-0.226206,-0.500108,-0.034462
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,-0.313031,-0.651260,0.441921,0.512923,-0.192007
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,-0.965521,0.164992,-0.163848,-0.500108,-0.192583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,-0.888006,1.056176,-0.300443,-0.500108,-0.192583
660,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,-0.752567,-0.809116,-0.077733,-0.094895,-0.116976
661,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,-0.532799,1.738550,-0.077733,-0.297501,-0.192391
662,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.157179,-0.918016,-0.659746,-0.500108,-0.048662


# Separando a base de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, class_dataset, test_size=0.25) # Vamos utilizar 75% dos dados para treinar a máquina e 25% para testar

In [ ]:
print(f'{X_train.shape[0]} dados para treinar o modelo')

498 dados para treinar o modelo


In [ ]:
print(f'{X_test.shape[0]} dados para testar o modelo')

166 dados para testar o modelo


# Rede Neural - MLP

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from random import shuffle

Vamos determinar a estratégia de divisão de validação cruzada, definindo o número de splits (dobras) para 10.

In [ ]:
cv = StratifiedKFold(n_splits=10)

Vamos realizar os testes com quantidades de neurônios e camadas ocultas diferentes.

In [ ]:
hidden_layer_sizes_1 = [(16, 8, 4), (32, 16, 8), (64, 32, 16), (36, 24, 36)]
hidden_layer_sizes_2 = [(8, 32, 16, 8), (16, 64, 32, 16), (32, 128, 64, 32)]
hidden_layer_sizes_3 = [(19, 57, 45, 19), (19, 38, 19), (19, 38, 25, 19)]

In [ ]:
mlp = MLP = MLPClassifier(random_state=7, alpha=1e-5)
mlp_param = dict()
mlp_param['solver'] =  ['lbfgs', 'adam', 'sgd']
mlp_param['activation'] = ['identity', 'tanh', 'logistic', 'relu']
mlp_param['hidden_layer_sizes'] = hidden_layer_sizes_1 + hidden_layer_sizes_2 + hidden_layer_sizes_3
mlp_param['learning_rate'] = ['constant', 'adaptive', 'invscaling']
mlp_max_n_iter = len(mlp_param['solver']) * len(mlp_param['activation']) * len(mlp_param['hidden_layer_sizes']) * len(mlp_param['learning_rate'])
print(f'Há {mlp_max_n_iter} combinações de parâmetros diferentes, porém, iremos executar o código explorando até 100 parametros')

Há 360 combinações de parâmetros diferentes, porém, iremos executar o código explorando até 100 parametros


In [ ]:
mlp_best_scores = (0, {}, None)
for i in range(10, 110, 10):
  search_mlp = RandomizedSearchCV(mlp, mlp_param, n_iter=i, scoring='accuracy', n_jobs=4, cv=cv, random_state=7)
  result_mlp = search_mlp.fit(X_train, y_train.values.ravel())
  print('=========Random Search Results for MLP==========')
  print('Best Score: %s' % result_mlp.best_score_)
  print('Best Hyperparameters: %s' % result_mlp.best_params_)
  print('\n')
  if result_mlp.best_score_ > mlp_best_scores[0]:
    mlp_best_scores = (result_mlp.best_score_, result_mlp.best_params_, result_mlp)

=========Random Search Results for MLP==========
Best Score: 0.867469387755102
Best Hyperparameters: {'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (19, 38, 19), 'activation': 'identity'}




/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


=========Random Search Results for MLP==========
Best Score: 0.8734693877551021
Best Hyperparameters: {'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (19, 38, 25, 19), 'activation': 'tanh'}




/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


=========Random Search Results for MLP==========
Best Score: 0.8755102040816327
Best Hyperparameters: {'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (19, 38, 19), 'activation': 'relu'}


=========Random Search Results for MLP==========
Best Score: 0.8795510204081631
Best Hyperparameters: {'solver': 'adam', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (19, 38, 25, 19), 'activation': 'identity'}


=========Random Search Results for MLP==========
Best Score: 0.8795510204081631
Best Hyperparameters: {'solver': 'adam', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (19, 38, 25, 19), 'activation': 'identity'}


=========Random Search Results for MLP==========
Best Score: 0.8795510204081631
Best Hyperparameters: {'solver': 'adam', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (19, 38, 25, 19), 'activation': 'identity'}


=========Random Search Results for MLP==========
Best Score: 0.8795510204081631
Best Hyperparameters: {'solver': 'adam', 'learning_

### Treinando o modelo com os parâmetros que obtiveram melhores resultados

In [ ]:
MLP = MLPClassifier(random_state=7, alpha=1e-5, solver = 'adam', learning_rate = 'invscaling', hidden_layer_sizes = (19, 38, 25, 19), activation = 'identity')
MLP.fit(X_train, y_train.values.ravel());
Y_test_prediction_MLP = MLP.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

print("\nAcuracia Rede Neural: Treinamento",  MLP.score(X_train, y_train)," Teste" ,MLP.score(X_test, y_test))
print("Clasification report:", classification_report(y_test, Y_test_prediction_MLP))
print("Confussion matrix:\n", confusion_matrix(y_test, Y_test_prediction_MLP))


Acuracia Rede Neural: Treinamento 0.8755020080321285  Teste 0.8373493975903614
Clasification report:               precision    recall  f1-score   support

           0       0.99      0.73      0.84        97
           1       0.72      0.99      0.83        69

    accuracy                           0.84       166
   macro avg       0.85      0.86      0.84       166
weighted avg       0.88      0.84      0.84       166

Confussion matrix:
 [[71 26]
 [ 1 68]]


# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn_param = dict()
knn_param['n_neighbors'] = range(4,11)
knn_param['weights'] = ['uniform', 'distance']
knn_param['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
knn_param['metric'] = ['manhattan', 'chebyshev', 'minkowski', 'euclidean']

In [ ]:
knn_best_scores = (0, {}, None)
for i in range(10, 110, 10):
  search_knn = RandomizedSearchCV(knn, knn_param, n_iter=i, scoring='accuracy', n_jobs=4, cv=cv, random_state=7)
  result_knn = search_knn.fit(X_train, y_train.values.ravel())
  print('=========Random Search Results for KNN==========')
  print('Best Score: %s' % result_knn.best_score_)
  print('Best Hyperparameters: %s' % result_knn.best_params_)

  if result_knn.best_score_ > knn_best_scores[0]:
    knn_best_scores = (result_knn.best_score_, result_knn.best_params_, result_knn)

=========Random Search Results for KNN==========
Best Score: 0.8553469387755103
Best Hyperparameters: {'weights': 'distance', 'n_neighbors': 9, 'metric': 'manhattan', 'algorithm': 'brute'}
=========Random Search Results for KNN==========
Best Score: 0.8553469387755103
Best Hyperparameters: {'weights': 'distance', 'n_neighbors': 9, 'metric': 'manhattan', 'algorithm': 'brute'}
=========Random Search Results for KNN==========
Best Score: 0.8553469387755103
Best Hyperparameters: {'weights': 'distance', 'n_neighbors': 9, 'metric': 'manhattan', 'algorithm': 'brute'}
=========Random Search Results for KNN==========
Best Score: 0.8553469387755103
Best Hyperparameters: {'weights': 'distance', 'n_neighbors': 9, 'metric': 'manhattan', 'algorithm': 'brute'}
=========Random Search Results for KNN==========
Best Score: 0.8573877551020408
Best Hyperparameters: {'weights': 'distance', 'n_neighbors': 7, 'metric': 'manhattan', 'algorithm': 'ball_tree'}
=========Random Search Results for KNN==========
Be

In [ ]:
KNN = KNeighborsClassifier(weights = 'distance', n_neighbors = 7, metric = 'manhattan', algorithm = 'ball_tree')
KNN.fit(X_train, y_train.values.ravel());
Y_test_prediction_KNN = KNN.predict(X_test)
print("\nAcuracia K-NN",  KNN.score(X_train, y_train)," Teste" ,KNN.score(X_test, y_test))
print(classification_report(y_test, Y_test_prediction_KNN))
print("Confussion matrix:\n", confusion_matrix(y_test, Y_test_prediction_KNN))


Acuracia K-NN 1.0  Teste 0.8192771084337349
              precision    recall  f1-score   support

           0       0.92      0.75      0.83        97
           1       0.72      0.91      0.81        69

    accuracy                           0.82       166
   macro avg       0.82      0.83      0.82       166
weighted avg       0.84      0.82      0.82       166

Confussion matrix:
 [[73 24]
 [ 6 63]]


 # SVM

In [ ]:
from sklearn.svm import SVC

svc = SVC()
param_svc = {'C': [0.1, 10, 20, 30, 40],
              'gamma': [0.01, 0.05, 0.07, 0.1, 'auto'],
              'kernel': ['rbf', 'linear', 'sigmoid','poly'],
              'decision_function_shape': ['ovo','ovr']
              }

In [ ]:
svc_best_scores = (0, {}, None)
for i in range(5, 55, 5):
  search_svc = RandomizedSearchCV(svc, param_svc, n_iter=i, scoring='accuracy', n_jobs=4, cv=cv, random_state=7)
  result_svc = search_svc.fit(X_train, y_train.values.ravel())
  print('=========Random Search Results for SVM Teste==========')
  print('Best Score: %s' % result_svc.best_score_)
  print('Best Hyperparameters: %s' % result_svc.best_params_)
  if result_svc.best_score_ > svc_best_scores[0]:
    svc_best_scores = (result_svc.best_score_, result_svc.best_params_, result_svc)

=========Random Search Results for SVM Teste==========
Best Score: 0.8715102040816326
Best Hyperparameters: {'kernel': 'rbf', 'gamma': 0.01, 'decision_function_shape': 'ovo', 'C': 30}
=========Random Search Results for SVM Teste==========
Best Score: 0.8735102040816326
Best Hyperparameters: {'kernel': 'linear', 'gamma': 0.01, 'decision_function_shape': 'ovo', 'C': 30}
=========Random Search Results for SVM Teste==========
Best Score: 0.8735102040816326
Best Hyperparameters: {'kernel': 'linear', 'gamma': 0.01, 'decision_function_shape': 'ovo', 'C': 30}
=========Random Search Results for SVM Teste==========
Best Score: 0.8735102040816326
Best Hyperparameters: {'kernel': 'linear', 'gamma': 0.01, 'decision_function_shape': 'ovo', 'C': 30}
=========Random Search Results for SVM Teste==========
Best Score: 0.8735102040816326
Best Hyperparameters: {'kernel': 'linear', 'gamma': 0.01, 'decision_function_shape': 'ovo', 'C': 30}
=========Random Search Results for SVM Teste==========
Best Score: 0

In [ ]:
SVC_ = SVC(kernel = 'linear', gamma = 0.01, decision_function_shape = 'ovo', C = 30)
SVC_.fit(X_train, y_train.values.ravel());
Y_test_prediction_SVC = SVC_.predict(X_test)
print("\nAcuracia SVC: Treinamento",  SVC_.score(X_train, y_train)," Teste" ,SVC_.score(X_test, y_test))
print(classification_report(y_test, Y_test_prediction_SVC))
print("Confussion matrix:\n", confusion_matrix(y_test, Y_test_prediction_SVC))


Acuracia SVC: Treinamento 0.8755020080321285  Teste 0.8253012048192772
              precision    recall  f1-score   support

           0       0.99      0.71      0.83        97
           1       0.71      0.99      0.82        69

    accuracy                           0.83       166
   macro avg       0.85      0.85      0.83       166
weighted avg       0.87      0.83      0.83       166

Confussion matrix:
 [[69 28]
 [ 1 68]]


# Ensemble Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

array = dataset.values
X = array[:,0:8]
Y = array[:,8]

# X = X_train.values
# Y = y_train.values.ravel()

clf_array = [SVC_]
for clf in clf_array:
    boosting_clf = AdaBoostClassifier(base_estimator=clf, random_state=7, n_estimators = 30, algorithm = 'SAMME')
    pima_scores = cross_val_score(clf, X, Y, cv=cv)
    boosting_scores = cross_val_score(boosting_clf, X, Y, cv=cv)    
    print('Media clf ',pima_scores.mean(),'\n', 'Desvio ',pima_scores.std(),'\n', 'Media Bagging',boosting_scores.mean(),'\n', 'Desvio ',boosting_scores.std())

Media clf  0.5752600633197649 
 Desvio  0.060632533691795186 
 Media Bagging 0.5737675260063321 
 Desvio  0.06208204271000622


In [ ]:
ABOOSTING = boosting_clf.fit(X_train, y_train.values.ravel())
Y_test_prediction_AB = ABOOSTING.predict(X_test)
print("\nAcuracia Boosting: Treinamento",  ABOOSTING.score(X_train, y_train)," Teste" ,ABOOSTING.score(X_test, y_test))
print(classification_report(y_test, Y_test_prediction_AB))
print("Confussion matrix:\n", confusion_matrix(y_test, Y_test_prediction_AB))


Acuracia Boosting: Treinamento 0.8694779116465864  Teste 0.8132530120481928
              precision    recall  f1-score   support

           0       0.90      0.76      0.83        97
           1       0.73      0.88      0.80        69

    accuracy                           0.81       166
   macro avg       0.81      0.82      0.81       166
weighted avg       0.83      0.81      0.81       166

Confussion matrix:
 [[74 23]
 [ 8 61]]


# Ensemble Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
clf_array = [(KNN, 'KNN'), (SVC_, 'SVC')]
for clf, name in clf_array:
    pima_scores = cross_val_score(clf, X, Y, cv=10)
    bagging_clf = BaggingClassifier(clf,max_samples=0.4, random_state=7)
    bagging_scores = cross_val_score(bagging_clf, X, Y, cv=10)    
    print(f'Media clf-{name} ',pima_scores.mean(), 'Desvio ',pima_scores.std(), 'Media Bagging',bagging_scores.mean(), 'Desvio ',bagging_scores.std())
    BAGGING = boosting_clf.fit(X_train, y_train.values.ravel())
    Y_test_prediction_BA = BAGGING.predict(X_test)
    print(f"\nAcuracia Bagging: Treinamento - {name}",  BAGGING.score(X_train, y_train)," Teste" ,BAGGING.score(X_test, y_test))
    print(classification_report(y_test, Y_test_prediction_BA))
    print("Confussion matrix:\n", confusion_matrix(y_test, Y_test_prediction_BA))
    print('\n\n')

Media clf-KNN  0.5301447308909996 Desvio  0.009549541210379367 Media Bagging 0.5528720036182724 Desvio  0.08226602743155902

Acuracia Bagging: Treinamento - KNN 0.8694779116465864  Teste 0.8132530120481928
              precision    recall  f1-score   support

           0       0.90      0.76      0.83        97
           1       0.73      0.88      0.80        69

    accuracy                           0.81       166
   macro avg       0.81      0.82      0.81       166
weighted avg       0.83      0.81      0.81       166

Confussion matrix:
 [[74 23]
 [ 8 61]]



Media clf-SVC  0.5752600633197649 Desvio  0.060632533691795186 Media Bagging 0.5691994572591587 Desvio  0.06178347247753602

Acuracia Bagging: Treinamento - SVC 0.8694779116465864  Teste 0.8132530120481928
              precision    recall  f1-score   support

           0       0.90      0.76      0.83        97
           1       0.73      0.88      0.80        69

    accuracy                           0.81       166
 

# Ensemble Voting

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold


kfold = StratifiedKFold(n_splits=10, random_state=7, shuffle = True)

estimators = [('SVC', SVC_), ('KNN', KNN), ('MLP', MLP)]

# create the ensemble model
voting = VotingClassifier(estimators)
results = cross_val_score(voting, X, Y, cv=kfold)
print(results.mean())

0.5768204432383537


In [ ]:
VOTING = voting.fit(X_train, y_train.values.ravel())
Y_test_prediction_VO = VOTING.predict(X_test)
print("\nAcuracia Voting: Treinamento",  VOTING.score(X_train, y_train)," Teste" ,VOTING.score(X_test, y_test))
print(classification_report(y_test, Y_test_prediction_VO))
print("Confussion matrix:\n", confusion_matrix(y_test, Y_test_prediction_VO))


Acuracia Voting: Treinamento 0.8795180722891566  Teste 0.8313253012048193
              precision    recall  f1-score   support

           0       0.99      0.72      0.83        97
           1       0.72      0.99      0.83        69

    accuracy                           0.83       166
   macro avg       0.85      0.85      0.83       166
weighted avg       0.87      0.83      0.83       166

Confussion matrix:
 [[70 27]
 [ 1 68]]


# Dificuldades encontradas
Nós tivemos que retirar os seguintes parâmetros do modelo SVM  
Best Hyperparameters: {'kernel': 'linear', 'gamma': 0.07, 'C': 1}  
Porque ao realizarmos a busca de melhores parâmetros, eles eram os que se sobressaiam sobre os outros, porém, no Ensemble Boosting e Bagging eles apresentavam inconsistência no resultado, chegando a precisão de 58% na classe 0 e 0% na classe 1. Não sabemos especificar o motivo deste problema.
